### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
url_built = f'{url}begin_date={begin_date}&end_date={end_date}&fl={field_list}&fq={filter_query}&sort={sort}&api-key={nyt_api_key}'


In [4]:
test_var = 0
while test_var < 10:
    print(test_var)
    test_var += 1

0
1
2
3
4
5
6
7
8
9


In [5]:
# Create an empty list to store the reviews
reviews_list = []

def save_to_list(response):
    if len(response['response']['docs']) > 0:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in response['response']['docs']:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f'Checked page {page}')
    else:
        # Print the page number that had no results then break from the loop
        print(f'No results for page {page}')

requestHeaders = {
    "Accept": "application/json"
}

# loop through pages 0-19
page = 0
while page < 20:
    # create query with a page number
    # API results show 10 articles at a time
    paginated_url = f'{url_built}&page={page}'

    try:
        # Make a "GET" request and retrieve the JSON
        reviews = requests.get(paginated_url, headers=requestHeaders).json()

        # Add a twelve second interval between queries to stay within API query limits
        time.sleep(12)

        # Try and save the reviews to the reviews_list
        save_to_list(reviews)
    except KeyError:
        # Check if it's a rate limit error
        try:
            errorcode = reviews['fault']['detail']['errorcode']
            if errorcode == 'policies.ratelimit.QuotaViolation':
                # Wait 1 minute and try again
                print('Rate limit reached. Pausing for 1 minute.')
                time.sleep(60)
                # Decrement page by 1 to try the same page again
                page -= 1
        # Possible KeyError from reviews['fault']['detail']['errorcode']
        except:
            print('Unhandled error')
            print(json.dumps(reviews, indent=4))
    page += 1


Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19
200


In [6]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[0:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [7]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df = pd.json_normalize(reviews_list)
df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [8]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

def extract_title(headline):
    start = headline.index('\u2018') + 1
    end = headline.index('\u2019')
    if ('\u2019 Review:' in headline):
        end = headline.index('\u2019 Review:')
    return headline[start:end]

df['title'] = df['headline.main'].apply(
    lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")]
)
df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [9]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df['keywords'] = df['keywords'].apply(extract_keywords)
df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [10]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

titles = df.title.to_list()
titles


['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [11]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie"
tmdb_key_string = f"?api_key={tmdb_api_key}"

In [12]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter == 50:
        time.sleep(1)   # Rate limit is 50/sec
        request_counter = 0

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    response = requests.get(f"{url}{tmdb_key_string}&query={title}").json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        id = response['results'][0]['id']

        # Make a request for a the full movie details
        details_url = f"https://api.themoviedb.org/3/movie/{id}{tmdb_key_string}"

        # Execute "GET" request with url
        details_response = requests.get(details_url).json()
        movie_details = details_response
        
        # Extract the genre names into a list
        genres = list(map(
            lambda genre: genre['name'],
            movie_details['genres']
        ))

        # Extract the spoken_languages' English name into a list
        spoken_languages = list(map(
            lambda lang: lang['english_name'],
            movie_details['spoken_languages']
        ))

        # Extract the production_countries' name into a list
        production_countries = list(map(
            lambda country: country['name'],
            movie_details['production_countries']
        ))

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            'title': title,
            'id': id,
            'genres': genres,
            'languages': spoken_languages,
            'countries': production_countries,
        })
        
        # Print out the title that was found
        print(f"Found {title}")
    except IndexError:
        print(f"{title} not found.")



Found The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo not found.
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Desi
F

In [13]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[0:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "id": 743040,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "languages": [
            "Spanish"
        ],
        "countries": [
            "Argentina"
        ]
    },
    {
        "title": "You Can Live Forever",
        "id": 887580,
        "genres": [
            "Drama",
            "Romance"
        ],
        "languages": [
            "English",
            "French"
        ],
        "countries": [
            "Canada",
            "United States of America"
        ]
    },
    {
        "title": "A Tourist\u2019s Guide to Love",
        "id": 813726,
        "genres": [
            "Romance",
            "Comedy"
        ],
        "languages": [
            "English",
            "Vietnamese"
        ],
        "countries": [
            "United States of America"
        ]
    },
    {
        "title": "Other People\u2019s Childr

In [14]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df.head()

,title,id,genres,languages,countries
0,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,887580,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist’s Guide to Love,813726,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People’s Children,803694,"[Drama, Comedy]","[English, French]",[France]
4,One True Loves,846961,"[Romance, Drama, Comedy]",[English],"[Germany, United States of America]"


### Merge and Clean the Data for Export

In [15]:
# Merge the New York Times reviews and TMDB DataFrames on title

# 	title	original_title	budget	genre	language	spoken_languages	homepage	overview	popularity	runtime	...	headline.main	headline.kicker	headline.content_kicker	headline.print_headline	headline.name	headline.seo	headline.sub	byline.original	byline.person	byline.organization
# 0	The Attachment Diaries	El apego	0	[Drama, Mystery, Thriller, Horror]	es	[Spanish]		Argentina, 1970s. A desperate young woman goes...	0.708	102	...	‘The Attachment Diaries’ Review: Love, Sick	None	None	The Attachment Diaries	None	None	None	By Jeannette Catsoulis	[{'firstname': 'Jeannette', 'middlename': None...	None
# 1	You Can Live Forever	You Can Live Forever	0	[Drama, Romance]	en	[French, English]	https://gooddeedentertainment.com/you-can-live...	When Jaime, a gay teenager, is sent to live in...	12.027	96	...	‘You Can Live Forever’ Review: Do You Love Me ...	None	None	You Can Live Forever	None	None	None	By Elisabeth Vincentelli	[{'firstname': 'Elisabeth', 'middlename': None...	None
# 2	One True Loves	One True Loves	0	[Romance, Drama, Comedy]	en	[English]		Emma and Jesse are living the perfect life tog...	28.872	100	...	‘One True Loves’ Review: A Romance Lost at Sea	None	None	One True Loves	None	None	None	By Brandon Yu	[{'firstname': 'Brandon', 'middlename': None, ...	None
# 3	The Lost Weekend: A Love Story	The Lost Weekend: A Love Story	0	[Documentary]	

merged_df = df.merge(tmdb_df, how='outer', on='title')
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,id,genres,languages,countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040.0,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo,NaN,NaN,NaN,NaN
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,887580.0,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love,813726.0,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children,803694.0,"[Drama, Comedy]","[English, French]",[France]


In [16]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'languages', 'countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[col].replace(char, '')

# Display the fixed DataFrame
merged_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,id,genres,languages,countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040.0,"['Drama', 'Mystery', 'Thriller', 'Horror']",['Spanish'],['Argentina']
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo,NaN,nan,nan,nan
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,887580.0,"['Drama', 'Romance']","['English', 'French']","['Canada', 'United States of America']"
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love,813726.0,"['Romance', 'Comedy']","['English', 'Vietnamese']",['United States of America']
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children,803694.0,"['Drama', 'Comedy']","['English', 'French']",['France']


In [17]:
# Drop "byline.person" column
merged_df = merged_df.drop('byline.person', axis='columns')

In [25]:
# Delete duplicate rows and reset index
# Delete duplicate rows based on "title" column
merged_df = merged_df\
    .drop_duplicates(subset='title')
# Reset index
merged_df = merged_df.reset_index(drop=True)
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,title,id,genres,languages,countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,743040.0,"['Drama', 'Mystery', 'Thriller', 'Horror']",['Spanish'],['Argentina']
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What’s Love Got to Do With It?’ Probably a Lo,NaN,nan,nan,nan
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None,You Can Live Forever,887580.0,"['Drama', 'Romance']","['English', 'French']","['Canada', 'United States of America']"
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None,A Tourist’s Guide to Love,813726.0,"['Romance', 'Comedy']","['English', 'Vietnamese']",['United States of America']
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None,Other People’s Children,803694.0,"['Drama', 'Comedy']","['English', 'French']",['France']


In [26]:
# Export data to CSV without the index
merged_df.to_csv("export.csv", index=False, header=True)